<a href="https://colab.research.google.com/github/AnuragSingla911/PythonStudy/blob/master/keras/House_Price_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import files

# Upload the file to Colab.
uploaded = files.upload()

Saving house_price_full+(1).csv to house_price_full+(1).csv


## Part 1: Import the Housing data and do feature transformations

In [3]:
df= pd.read_csv('house_price_full+(1).csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [4]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [5]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [6]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [7]:
from tensorflow import keras

In [8]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [9]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1)
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [10]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 [==============================] - 0s 1ms/step - loss: 101.2158
Epoch 2/10
16/16 [==============================] - 0s 1ms/step - loss: 26.4925
Epoch 3/10
16/16 [==============================] - 0s 1ms/step - loss: 5.8488
Epoch 4/10
16/16 [==============================] - 0s 1000us/step - loss: 1.5770
Epoch 5/10
16/16 [==============================] - 0s 1ms/step - loss: 0.7106
Epoch 6/10
16/16 [==============================] - 0s 1ms/step - loss: 0.4954
Epoch 7/10
16/16 [==============================] - 0s 1ms/step - loss: 0.4106
Epoch 8/10
16/16 [==============================] - 0s 1ms/step - loss: 0.3548
Epoch 9/10
16/16 [==============================] - 0s 1ms/step - loss: 0.3137
Epoch 10/10
16/16 [==============================] - 0s 1ms/step - loss: 0.2809


In [11]:
model.predict(X)[:,0]

16/16 [==============================] - 0s 948us/step


array([13.125479 , 13.137537 , 13.102415 , 13.0996   , 13.224314 ,
       12.712591 , 12.676264 , 13.21248  , 13.0819435, 13.230318 ,
       13.111151 , 13.209061 , 13.086105 , 13.134905 , 13.139842 ,
       13.130783 , 13.116627 , 13.205894 , 13.124334 , 13.131535 ,
       13.089823 , 13.23129  , 13.109575 , 13.212158 , 13.229198 ,
       13.095957 , 13.084861 , 13.22044  , 13.140089 , 13.074783 ,
       13.229759 , 13.210047 , 13.226061 , 13.210699 , 13.196866 ,
       13.046915 , 12.718673 , 13.10561  , 13.10121  , 13.083612 ,
       13.116238 , 13.230875 , 12.714875 , 13.108785 , 12.687191 ,
       13.132286 , 13.100807 , 13.102415 , 13.1281395, 13.202486 ,
       13.21248  , 13.061468 , 13.070099 , 13.107597 , 13.141579 ,
       13.224607 , 13.220591 , 13.137502 , 13.12624  , 12.654138 ,
       13.202486 , 13.071381 , 13.212158 , 13.077322 , 13.209061 ,
       12.711065 , 13.113506 , 13.137878 , 12.567932 , 13.091467 ,
       13.215343 , 13.064073 , 13.219223 , 13.103615 , 13.1258